# Unit tests

<img src='unittest_success.gif'>

In [ ]:
#| export
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import patch
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from typing import Union
import pandas as pd
import subprocess
import random
import shutil
import json
import os

from pyliwc import Liwc

In [ ]:
#| hide

# unit tests

def test_analyze_csv_creates_output_file(input_file, output_file):
    liwc = Liwc('LIWC-22-cli.exe')
    # Ensure the output file does not exist before running the function
    if os.path.exists(output_file):
        os.remove(output_file)
    
    # Call the function to be tested
    liwc.analyze_csv(input_file=input_file, output_location=output_file, row_id_indices="1", column_indices="3")
    
    # Use `test` from fastcore.test to check if the output file was created
    test_eq(os.path.exists(output_file), True)
    os.remove(output_file)

# Define the input and output paths
input_file = r"../data/US-president.csv"
output_file = rf"{random.randint(0, 999)}_output.csv"

# Run the test function
test_analyze_csv_creates_output_file(input_file, output_file)


In [ ]:
#| hide

def test_analyze_df_returns_dataframe(input_csv_path):
    """
    Test that the analyze_df function returns a DataFrame with expected properties.
    
    """
    liwc = Liwc('LIWC-22-cli.exe')
    # Load the test DataFrame
    df_test = pd.read_csv(input_csv_path)
    
    # Call the function to be tested
    result_df = liwc.analyze_df(df_test)
    
    # Check that the return value is a DataFrame
    test_eq(type(result_df), pd.core.frame.DataFrame)

# Define the input CSV path
input_csv_path = r"../data/US-president.csv"

# Run the test function
test_analyze_df_returns_dataframe(input_csv_path)


In [ ]:
#| hide

def test_analyze_folder_creates_output_file(input_folder, output_location):
    """
    Test that the analyze_folder function creates an output file in the specified output folder.
    """
    liwc = Liwc('LIWC-22-cli.exe')
    # Remove all files in the specified directory
    [os.remove(os.path.join(output_location, filename)) for filename in os.listdir(output_location) if os.path.isfile(os.path.join(output_location, filename))]

    # Call the function to be tested
    liwc.analyze_folder(input_folder=input_folder, output_location=output_location)
    
    # Use `test_eq` from fastcore.test to check if the output file was created
    test_eq(os.path.exists(any(filename.endswith('.csv') for filename in os.listdir(output_location))), True)
    # Clean up by removing the output file
    [os.remove(os.path.join(output_location, filename)) for filename in os.listdir(output_location) if os.path.isfile(os.path.join(output_location, filename))]


# Define the input folder and output folder paths
input_folder = r"../data/inaugural-address"
output_location = r"../data/unittests/"

# Run the test function
test_analyze_folder_creates_output_file(input_folder, output_location)


In [ ]:
#| hide

def test_analyze_string_creates_output_file(input_string, output_location, liwc_dict='LIWC22'):
    """
    Test that the analyze_string function creates an output file at the specified location.
    
    """
    liwc = Liwc('LIWC-22-cli.exe')
    output_file = os.path.join(output_location, 'WC-result.csv')
    # Ensure the output file does not exist before running the function
    if os.path.exists(output_file):
        os.remove(output_file)
    
    # Call the function to be tested
    liwc.analyze_string(input_string=input_string, output_location=output_file, liwc_dict=liwc_dict)
    
    # Use `test_eq` from fastcore.test to check if the output file was created
    test_eq(os.path.exists(output_file), True)
    
    # Clean up by removing the output file
    os.remove(output_file)

# Define the input string and output file path
input_string = "This is a test string for LIWC analysis."
output_location = r"../data/"

# Run the test function
test_analyze_string_creates_output_file(input_string, output_location)


Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8


Processing input text...


Done. Please examine results in C:\Users\clacan\OneDrive\Etude\AP\_Notebooks\_Git\PyLIWC\unitest\..\data\WC-result.csv


In [ ]:
#| hide

def test_analyze_string_to_json_returns_dict(input_string, expected_keys, liwc_dict='LIWC22'):
    """
    Test that the analyze_string_to_json function

    """
    liwc = Liwc('LIWC-22-cli.exe')
    # Call the function to be tested
    result_dict = liwc.analyze_string_to_json(input_string=input_string, liwc_dict=liwc_dict)
    
    # Check that the return value is a dictionary
    test_eq(type(result_dict), dict)
    
    # Check that the dictionary contains the expected keys
    test_eq(set(result_dict.keys()) >= expected_keys, True)

    test_eq(len(result_dict), 119)

# Define the input string and expected keys in the result dictionary
input_string = "This is a test string for LIWC analysis."
expected_keys = {'WC', 'Analytic', 'Clout', 'Authentic', 'Tone'}  # Replace with actual expected keys

# Run the test function
test_analyze_string_to_json_returns_dict(input_string, expected_keys)

Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8


In [ ]:
#| hide

# unit tests

def test_analyze_lsm_returns_dataframe(input_csv_path):
    """
    Test that the analyze_lsm function returns a DataFrame or dictionary with expected properties.
    
    """
    liwc = Liwc('LIWC-22-cli.exe')
    
    # Load the test DataFrame
    df_test = pd.read_csv(input_csv_path)
    
    # Call the function to be tested
    result = liwc.analyze_lsm(df_test,
                              calculate_lsm="pairwise",  
                              person_column='President',
                              group_column='Party',
                              text_column='Text',
                              omit_speakers_word_count=1)
    
    test_eq(isinstance(result, dict), True)

    # Check the keys in the result dictionary
    if 'person_level' in result:
        test_eq(isinstance(result['person_level'], pd.DataFrame), True)
    if 'group_level' in result:
        test_eq(isinstance(result['group_level'], pd.DataFrame), True)


# Define the input CSV path
input_csv_path = r"../data/US-president.csv"


# Run the test function
test_analyze_lsm_returns_dataframe(input_csv_path)

In [ ]:
#| hide

# unit test

def test_narrative_arc_returns_dataframe(input_csv_path):
    """
    Test that the narrative_arc function returns a DataFrame or dictionary with expected properties.
    
    """

    liwc = Liwc('LIWC-22-cli.exe')
    
    # Load the test DataFrame
    df_test = pd.read_csv(input_csv_path)
    
    # Call the function to be tested
    result = liwc.narrative_arc(
        df=df_test, 
        column_names=['Text'], 
        output_individual_data_points=True, 
        scaling_method='0-100', 
        segments_number=5, 
        skip_wc=10
    )
    
    test_eq(isinstance(result, pd.DataFrame), True)

# Define the input CSV path
input_csv_path = r"../data/US-president.csv"


# Run the test function
test_narrative_arc_returns_dataframe(input_csv_path)


In [ ]:
#| hide

# unit tests for figure

def test_plot_narrative_arc_returns_figure(input_csv_path):
    import matplotlib.pyplot as plt
    """
    Test that the plot_narrative_arc function returns a figure.
    """
    
    liwc = Liwc('LIWC-22-cli.exe')
    
    # Load the test DataFrame
    df_test = pd.read_csv(input_csv_path)
    
    # Perform the narrative arc analysis
    narrative_results = liwc.narrative_arc(
        df=df_test, 
        column_names=['Text'], 
        output_individual_data_points=True, 
        scaling_method='0-100', 
        segments_number=5, 
        skip_wc=10
    )
    
    # Call the function to be tested
    fig = liwc.plot_narrative_arc(narrative_results)
    
    # Check if the returned object is a matplotlib figure
    test_eq(isinstance(fig, plt.Figure), True)

# Define the input CSV path
input_csv_path = r"../data/US-president.csv"

# Run the test function
test_plot_narrative_arc_returns_figure(input_csv_path)